In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

# Reading the dataset and Bay of Bengal South Boundary Points File
ds = xr.open_dataset(r"G:\MJI\SKZ\Script\MSP_Data\BoB_South_Boundary_SST\Input_File\ERA5_2019\SST_Hourly_2019.nc")
df = pd.read_excel(r"G:\MJI\SKZ\Script\MSP_Data\BoB_South_Boundary_SST\Input_File\BoB_South_Boundary_Line_Points.xlsx")

# Defining a dataframe to store the extracted data
ts_df = pd.DataFrame()

# Iterating through all the points and extracting nearest grid point data from the dataset
for lat, lon, point_id in zip(df["Lat"], df["Lon"], df["Point_ID"]):

    sst_data = ds["sst"].sel(longitude = lon, latitude = lat, method="nearest")
    grid_lon = sst_data["longitude"].data
    grid_lat = sst_data["latitude"].data
    
    point_sst_df = pd.DataFrame( {'Time': ds["sst"].time.data, 
                                  '{}_{}_{}'.format(point_id, grid_lat, grid_lon ): sst_data.data} )

    if point_id == 1:
        ts_df = pd.concat([ts_df, point_sst_df])

    else:
        ts_df = ts_df.merge(point_sst_df, on = 'Time')

# Saving the extracted merged dataframe as csv file
ts_df.to_csv(r"ERA5_Hourly_Sea_Surface_Temperature_in_Kelvin_2019.csv", index = False, float_format = "%.4f")